In [ ]:
# Helper: set WEIGHT_SOURCE to either a public HTTP(s) URL or a model identifier like 'yolov12n.pt'.
# If it's a URL the cell downloads the file with progress. If it's a model identifier, the later
# YOLO(...) call will attempt to download/use it via the ultralytics API which is often fastest.
import os
WEIGHT_SOURCE = "yolov12n.pt"  # set to a direct URL (https://...) or to a model name like 'yolov12n.pt'
WEIGHT_PATH = 'yolov12n.pt'

if isinstance(WEIGHT_SOURCE, str) and WEIGHT_SOURCE.lower().startswith('http'):
    if not os.path.exists(WEIGHT_PATH):
        print('Downloading weights from', WEIGHT_SOURCE)
        try:
            import requests
            from tqdm import tqdm
            resp = requests.get(WEIGHT_SOURCE, stream=True)
            total = int(resp.headers.get('content-length', 0))
            with open(WEIGHT_PATH, 'wb') as f:
                for chunk in tqdm(resp.iter_content(chunk_size=8192), total=max(1, total//8192)):
                    if chunk:
                        f.write(chunk)
            print('Download complete:', WEIGHT_PATH)
        except Exception as e:
            print('Download failed:', e)
    else:
        print('Weights already present at', WEIGHT_PATH)
else:
    print('Using model identifier:', WEIGHT_SOURCE)
    print('When the notebook calls YOLO(WEIGHT_SOURCE) the ultralytics loader will attempt to fetch or use this identifier.')
    # Do not attempt to download here; rely on ultralytics to fetch it efficiently in the runtime.


# Kaggle: YOLOv12 Experiments (yolov12-n quick start)

This notebook installs dependencies, downloads yolov12-n weights with progress, runs inference on a test sample from the project, and shows a small threshold tuning grid. It also documents how to sync results back to your local project folder.

## 1) Setup Kaggle Notebook Environment
Select GPU accelerator in Notebook Settings -> Accelerator -> GPU.
Run the following cells to verify GPU and Python environment.

In [ ]:
# Verify GPU availability and basic environment
import os
# Show NVIDIA status
!nvidia-smi -L
# Verify PyTorch sees GPU
import torch
print('torch version:', torch.__version__)
print('cuda available:', torch.cuda.is_available())
print('cuda devices:', torch.cuda.device_count())

## 2) Clone or sync the project repository
You can either `git clone` your repository, use Kaggle Datasets, or upload a ZIP.
Examples below assume you `git clone` from GitHub. If you prefer uploading your project as a Kaggle Dataset, use the Kaggle UI or `kaggle datasets` CLI.

In [ ]:
# Example: clone your repo (replace URL)
!git clone https://github.com/youruser/yourrepo.git project || true
!ls -la

## 3) Install YOLOv12 and Python dependencies
Install GPU-enabled PyTorch first (match the CUDA version shown by `nvidia-smi`). Then install ultralytics or yolov12 source if available.

In [ ]:
# Example installs (adjust CUDA wheel as needed)
# Install a compatible torch + torchvision (example for CUDA 11.8)
!pip install -U pip
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 || true
# Install ultralytics (works with YOLOv8 and newer APIs). If yolov12 has a package, install that instead.
!pip install -U ultralytics==8.0.0 || true
# If yolov12 is published or you have a repo, clone and install it: (example placeholder)
# !git clone https://github.com/yourorg/yolov12.git && cd yolov12 && pip install -e .
import ultralytics, torch
print('ultralytics', ultralytics.__version__)
print('torch', torch.__version__, 'cuda available', torch.cuda.is_available())

## 4) Download yolov12-n weights (with progress)
If a public URL is available for yolov12-n weights, use `wget` or a Python stream with `tqdm` to show progress.

In [ ]:
# Example: download with wget (replace URL)
WEIGHT_URL = 'https://example.com/yolov12n.pt'
WEIGHT_PATH = 'yolov12n.pt'
# If you have a URL, uncomment the line below: wget -q --show-progress will display progress in Kaggle
# !wget --progress=bar:force -O {WEIGHT_PATH} {WEIGHT_URL}
print('If you have a direct URL to yolov12-n weights, download it to', WEIGHT_PATH)

## 5) Prepare dataset and single test sample
Place one test video/image in a folder and optionally create a small YAML describing paths. For inference, labels are not required.

In [ ]:
# Example: copy one test image from project folder if present
# Adjust path to where you cloned the repo above (project)
import os
src = 'project/Test samples/3.mp4'
dst = '/kaggle/working/test_sample_3.mp4'
if os.path.exists(src):
    !cp -v 
 
 || true
print('Test sample at', dst)

## 6) Run inference with yolov12-n on the test sample
Use either the repo's detect CLI or the Python API. Below shows Python-style usage with ultralytics API (works for many YOLO versions).

In [ ]:
# Python API example (ultralytics-style)
from ultralytics import YOLO
# Replace with yolov12 weight path or a local file you uploaded
weights = 'yolov12n.pt'  # change if using another path
model = YOLO(weights)
# Run a single inference and save results to /kaggle/working/runs
res = model.predict(source='/kaggle/working/test_sample_3.mp4', conf=0.25, imgsz=640, save=True)
print('Saved results, sample output object:', type(res))

## 7) Tune Confidence / IoU thresholds on one test sample
Loop over a small grid of `conf` and `iou` values, run inference, and display side-by-side results. Save annotated outputs for comparison.

In [ ]:
from IPython.display import display, Image, HTML
import os
from ultralytics import YOLO
weights = 'yolov12n.pt'
model = YOLO(weights)
confs = [0.1, 0.25, 0.5]
ious = [0.45, 0.5, 0.6]
outputs = []
for conf in confs:
    for iou in ious:
        name = f'test_conf_{conf}_iou_{iou}'.replace('.', '_')
        print('Running', name)
        model.predict(source='/kaggle/working/test_sample_3.mp4', conf=conf, iou=iou, imgsz=640, save=True, project='/kaggle/working/runs', name=name)
        outputs.append((conf,iou,'/kaggle/working/runs',name))
print('Done tuning grid. Outputs saved under /kaggle/working/runs')

## 8) Save, export and sync results back to local project folder
Options:
- Commit and push results to GitHub from Kaggle (requires repo access token)
- Use `kaggle datasets create -p /kaggle/working/runs` to publish artifacts and then download locally
- Use `rsync` or `scp` from the notebook VM to your machine (if your machine is reachable)
Example: produce a ZIP and download via the Kaggle UI or `wget` from a public location.

In [ ]:
# Example: zip results for download or upload as a Kaggle dataset
!zip -r runs_artifacts.zip /kaggle/working/runs || true
print('Created runs_artifacts.zip')

## 9) Reproducibility and quick notes
Set seeds and deterministic flags to make results more reproducible. Note that first inference may be slower due to model/AOT warmup and weight download. yolov12-n is lightweight and should be ready quickly on a GPU instance.

In [ ]:
import random, numpy as np, torch
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)
print('Seeds set')

## 10) Benchmarking: GPU timing and throughput
Measure per-frame inference time and GPU memory usage for simple benchmarking.

In [ ]:
import time
from ultralytics import YOLO
weights = 'yolov12n.pt'
model = YOLO(weights)
# Warm up
_ = model.predict(source='/kaggle/working/test_sample_3.mp4', conf=0.25, imgsz=640, stream=True)
# Simple timing example (single image approach would be faster to stabilize)
print('Run a small timing loop in your environment to get FPS and memory metrics')

## 11) Simple automated check for inference output
A small test that checks an output file exists after running inference. You can adapt this into a pytest file.

In [ ]:
# Quick check: ensure at least one annotated file exists after running detection
import glob
outs = glob.glob('/kaggle/working/runs/**/*.*', recursive=True)
print('Found output files count:', len(outs))

In [ ]:
# Package results for download/publishing
import os
from pathlib import Path
runs_dir = Path('/kaggle/working/runs') if os.path.exists('/kaggle/working/runs') else Path('runs')
zip_path = 'runs_artifacts.zip'
if runs_dir.exists():
    print('Zipping', runs_dir, '->', zip_path)
    !zip -r {zip_path} {runs_dir} || true
    print('Zipped artifacts to', zip_path)
else:
    print('No runs directory found at', runs_dir)


In [ ]:
# Diagnostic cell: check ultralytics installation and specific module
import sys, importlib, importlib.util, os
print('Python:', sys.version.split()[0])

try:
    import ultralytics
    print('ultralytics imported, version:', getattr(ultralytics, '__version__', 'unknown'))
    print('ultralytics package file:', getattr(ultralytics, '__file__', 'unknown'))
except Exception as e:
    print('ultralytics import failed:', repr(e))

# Check whether the problematic submodule can be resolved
mod_name = 'ultralytics.utils.callbacks.hub'
spec = importlib.util.find_spec(mod_name)
print(mod_name, 'spec:', spec)

# Show list of available ultralytics.utils.callbacks submodules if package present
try:
    import pkgutil
    import ultralytics.utils.callbacks as _cbs
    submods = [name for _, name, _ in pkgutil.iter_modules(_cbs.__path__)]
    print('ultralytics.utils.callbacks submodules:', submods)
except Exception as e:
    print('Could not list ultralytics.utils.callbacks submodules:', repr(e))

# Check weight filename suggestions and existence in working dirs
candidates = ['yolov12n.pt', 'yolo12n.pt', 'yolov12-n.pt', 'yolov12-n.pt']
for c in candidates:
    for p in [c, os.path.join('/kaggle/working', c), os.path.join('/kaggle/input', c)]:
        if os.path.exists(p):
            print('Found weight file:', p)

print('\nIf the module is missing, run a pip install/upgrade cell (see next cell in the notebook or run: pip install -U ultralytics).')
